In [1]:
%load_ext autoreload
%autoreload explicit

from itertools import chain
from collections import Counter
from functools import cache
import re
from pathlib import Path


from prelib import data_dir
%aimport e6db.utils
from e6db.utils import TagSetNormalizer, tag_categories, tag_category2id

# Create the mappings

By default, the tag strings in this dataset are using the raw e621 format, with underscores. `TagNormalizer` enables renaming the tags and adding new spelling as aliases.

The tag formating choices made here are specific to [this dataset](https://huggingface.co/datasets/k4d3/furry) meant for PDXL training with sd-scripts. You should adapt it to your own dataset, model and trainer.

In [2]:
cat_artist = e6db.utils.tag_category2id['artist']
cat_lore = e6db.utils.tag_category2id['lore']

tagset_normalizer = e6db.utils.TagSetNormalizer(data_dir)
tagid2cat = tagset_normalizer.tag_normalizer.tag_categories

@cache
def tag_mapfun(tag_underscores, tid):
    """
    Maps raw e621 tags to more natural forms.
    Will be run on:

    * The list of output tag strings,
    * Keys from the dictionary mapping strings to ids, contains canonical tag and aliases,
    * Implication source tags that are not used frequently enough to get an id.

    Returns a list, where the first string is the canonical tag used in the output,
    the others are additional aliases used for recognizing the tag.
    """
    cat = tagid2cat[tid] if tid is not None else -1
    tag = tag_underscores.replace('_', ' ')
    tags = [tag, tag_underscores]
    if cat == cat_artist:
        if not tag.startswith('by '):
            # 'by ' is used in the output tags
            tags.insert(0, f'by {tag.removesuffix(' (artist)')}')
        if not tag.endswith('(artist)'):
            artist = tag.removeprefix('by ')
            tags.append(f'{artist} (artist)')
    elif cat == cat_lore and not tag.endswith(' (lore)'):
            tags.append(f'{tag} (lore)')

    escaped_parens = [t.replace('(', r'\(').replace(')', r'\)') for t in tags]
    for t, ep in zip(tags[1:], escaped_parens[1:]):
        if t != ep:
            tags.append(ep)
    if escaped_parens[0] != tags[0]:
        tags.insert(0, escaped_parens[0]) # apparently sd-scripts require escaped parentheses 
    if ':' in tag: # Recognize tags where ':' were replaced by a space
        tags.append(tag.replace(':', ' '))
    return tags


# Note: uses a single function to map both tag recognition and tags in the output, but this will change
tagset_normalizer = tagset_normalizer.map_tags(tag_mapfun, on_conflict="overwrite_rarest") 
# on_conflict: use warn to debug conflicts. silent, overwrite, overwrite_rarest, warn, raise
del tag_mapfun # drop the cache

# Add some underscores back
tag_normalizer = tagset_normalizer.tag_normalizer
tag_normalizer.rename_output('rating explicit', 'rating_explicit')
tag_normalizer.rename_output('rating questionable', 'rating_questionable')
tag_normalizer.rename_output('rating safe', 'rating_safe')
# Custom mappings
tag_normalizer.add_input_mappings('explicit', 'rating_explicit')
tag_normalizer.add_input_mappings('score_explicit', 'rating_explicit')
tag_normalizer.add_input_mappings('safe', 'rating_safe', on_conflict='overwrite')
tag_normalizer.add_input_mappings('score_safe', 'rating_safe')
tag_normalizer.add_input_mappings('questionable', 'rating_questionable', on_conflict='overwrite')
tag_normalizer.add_input_mappings('score_questionable', 'rating_questionable')

In [3]:
tagset_normalizer.map_tags?

Signature:
tagset_normalizer.map_tags(
    mapfun: Callable[[str, int | None], str | list[str]],
    map_input=True,
    map_output=True,
    on_conflict='raise',
) -> 'TagSetNormalizer'
Docstring:
Apply a function to all tag strings.

The provided function will be run on:

* The of list output tag strings,
* Keys from the dictionary mapping strings to ids, contains canonical
  tag and aliases,
* Implication source tags that are not used frequently enough to get an
  id assigned (less than twice).

The function should return a list, where the first string is the
canonical tag used in the output, the others are additional aliases
used for recognizing the tag.
File:      ~/code/e6db/e6db/utils/__init__.py
Type:      method

# Edit caption files

Loads, normalize and remove implied tags from a files in a sd-script hierarchy of directories.

Unknown tags are kept untouched.

Specific to kade's format, captions are detected as tags ending with a period and are moved to the end of the file.

In [4]:
# We will use tagset_normalizer.encode()/decode() to normalize tags. Demo:
tags, implied = tagset_normalizer.encode(['solo', 'male', 'canine', 'mammal', 'unknown tag'])
print(f'{tags=} {implied=}')

tagset_normalizer.decode(tags)

tagset_normalizer.encode?

tags=[6, 5, 17, 'unknown tag'] implied={0, 15}


Signature: tagset_normalizer.encode(tags: Iterable[str], keep_implied=False)
Docstring:
Encode a list of string as numerical ids and strip implied tags.

Unknown tags are returned as strings.

Returns :

* a list of tag ids and unknown tag strings,
* a list of implied tag ids.
File:      ~/code/e6db/e6db/utils/__init__.py
Type:      method

In [5]:
RE_SEP = re.compile(r'[,\n]') # Split on commas and newline

dataset_root = Path('~/repos/kade/furry').expanduser()
output_dir = Path('/tmp/furry_fix')
#output_dir = dataset_root # clobber mode

def load_caption(fp):
    tags, captions = [], []
    with open(fp, 'rt') as fd:
        for chunk in RE_SEP.split(fd.read()):
            chunk = chunk.strip()
            if not chunk:
                continue
            if chunk.endswith('.'):
                captions.append(chunk)
            else:
                tags.append(chunk)
    return tags, captions


counter = Counter()
for file in chain(dataset_root.glob('**/*.txt'), dataset_root.glob('**/*.cap*')):
    if 'sample-prompts' in file.name:
        continue
    tags, captions = load_caption(file)
    orig_tags = tags
    tags, implied = tagset_normalizer.encode(tags)
    counter.update(tags)
    tags = tagset_normalizer.decode(tags)
    if tags == orig_tags:
        continue

    output_file = output_dir / file.relative_to(dataset_root)
    output_file.parent.mkdir(parents=True, exist_ok=True)
    result = ', '.join(chain(tags, captions))
    with open(output_file, 'wt') as fd:
        fd.write(result)

In [6]:
# Shows tag frequencies in the dataset, not counting the occurrences of implied tags
for tag, count in counter.most_common(n=100):
    if isinstance(tag, int):
        tag_string = tagset_normalizer.tag_normalizer.decode(tag)
        cat = tag_categories[tagset_normalizer.tag_normalizer.tag_categories[tag]]
        print(f'{tag_string:<30} {count=} (e621:{cat})')
    else:
        print(f'{tag:<30} {count=} (unknown)')

solo                           count=10877 (e621:general)
rating_explicit                count=9363 (e621:general)
anthro                         count=7274 (e621:general)
nude                           count=5894 (e621:general)
female                         count=4525 (e621:general)
hi res                         count=4378 (e621:meta)
digital media \(artwork\)      count=4245 (e621:meta)
erection                       count=3952 (e621:general)
male                           count=3821 (e621:general)
looking at viewer              count=3600 (e621:general)
duo                            count=3555 (e621:general)
balls                          count=3496 (e621:general)
tail                           count=3485 (e621:general)
nipples                        count=3474 (e621:general)
open mouth                     count=3264 (e621:general)
smile                          count=3089 (e621:general)
blush                          count=2956 (e621:general)
english text                   count

In [7]:
# Top 100 of unknown tags:
print(', '.join([f'{k}({v})' for k,v in counter.most_common() if isinstance(k, str)][:100]))

furry male(1511), feet out of frame(306), furry sticker(306), furry with furry(290), animal focus(287), male pubic hair(251), furry with non-furry(243), wolf boy(243), gynomorph female(224), intersex intersex(140), male masturbation(125), gynomorph male(118), armpits(116), navel hair(107), by spaceengine(94), gynomorph gynomorph(91), male ambiguous(89), fox girl(87), tiger boy(87), photo \(medium\)(78), score explicit(63), clothed nude(60), greg rutkowski(54), five nights at freddy's security breach(53), blp(52), avery palmer(50), hamgas(48), bare pectorals(47), digimon \(creature\)(47), cum on boy(39), two-tone skin(39), non- breasts(35), animal hands(34), black male underwear(29), by jwst(29), leopard ears(28), non- balls(28), ground vehicle(27), cum on pectorals(25), white male underwear(25), female ambiguous(25), fox boy(24), by hubble(24), rabbit girl(22), andromorph male(22), painting \(medium\)(22), demon horns(19), herm male(19), webcomic character(19), lion boy(18), herm femal